In [45]:
import os
import pickle
import tsplib95
import csv
import time
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms.approximation.traveling_salesman import *

In [46]:
def get_tsp_graph(name) -> nx.Graph:
    # load from pkl file if exists
    graph_file_path = os.path.join("tsp_dataset", f"{name}.graph.pkl")
    if os.path.exists(graph_file_path):
        # print(f"Loading {name} from pkl {graph_file_path}")
        with open(graph_file_path, "rb") as file:
            G = pickle.load(file)
        return G.to_undirected()

    tsp_file_path = os.path.join("tsp_dataset", f"{name}.tsp")
    # print(f"Loading {name} from tsplib {tsp_file_path}")
    problem = tsplib95.load(tsp_file_path)
    G = problem.get_graph()

    # remove redundant edges
    if problem.edge_weight_type == "EXPLICIT" and problem.edge_weight_format == "FULL_MATRIX":
        for u in G.nodes:
            for v in G.nodes:
                if u > v:
                    G.remove_edge(u, v)

    # remove self loops
    loop_edges = list(nx.selfloop_edges(G))
    G.remove_edges_from(loop_edges)

    # save as pkl file
    with open(graph_file_path, "wb") as file:
        print(f"Saving {name} to pkl {graph_file_path}")
        pickle.dump(G, file)

    return G

In [47]:
tsp_dataset_file_path = os.path.join("tsp_dataset", "tsp_dataset.csv")
print(f"Loading tsp dataset from {tsp_dataset_file_path}")
tsp_dataset = pd.read_csv(tsp_dataset_file_path)

Loading tsp dataset from tsp_dataset\tsp_dataset.csv


In [48]:
def get_opt_tour_length(name):
    return tsp_dataset[tsp_dataset["name"] == name]["opt_tour_length"].values[0]

In [49]:
def get_tour_length(G: nx.graph, tour: list):
    tour_length = 0
    for i in range(len(tour) - 1):
        u, v = tour[i], tour[i + 1]
        w = G[u][v]["weight"]
        tour_length += w
    return tour_length

In [50]:
def get_apx_ratio(G: nx.graph, tour: list):
    return get_tour_length(G, tour) / get_opt_tour_length(G.name)

In [51]:
tsp_dataset

,name,dimension,opt_tour_length,type,comment,edge_weight_type,edge_weight_format
0,burma14,14,3323,TSP,14-Staedte in Burma (Zaw Win),GEO,FUNCTION
1,ulysses16,16,6859,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,NaN
2,gr17,17,2085,TSP,17-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
3,gr21,21,2707,TSP,21-city problem (Groetschel),EXPLICIT,LOWER_DIAG_ROW
4,ulysses22,22,7013,TSP,Odyssey of Ulysses (Groetschel/Padberg),GEO,NaN
...,...,...,...,...,...,...,...
91,u2319,2319,234256,TSP,Drilling problem (Reinelt),EUC_2D,NaN
92,pr2392,2392,378032,TSP,2392-city problem (Padberg/Rinaldi),EUC_2D,NaN
93,pcb3038,3038,137694,TSP,Drilling problem (Junger/Reinelt),EUC_2D,NaN
94,fnl4461,4461,182566,TSP,Die 5 neuen Laender Deutschlands (Ex-DDR) (Bac...,EUC_2D,NaN


In [52]:
from mst import MST
st = MST()

In [53]:
def get_total_edge_weight(G: nx.graph):
    total_edge_weight = 0
    for u, v in G.edges:
        total_edge_weight += G[u][v]["weight"]
    return total_edge_weight

In [54]:
from simulated_annealing import SimulateAnnealing

In [55]:
with open("sa_results.csv", "a", newline="") as csvfile:
    writer = csv.writer(csvfile)

    for name in tsp_dataset["name"]:
        print(name)

        G = get_tsp_graph(name)
        dimension = G.number_of_nodes()
        opt_tour_length = get_opt_tour_length(name)

        d = {
            "name": name,
            "dimension": dimension,
            "opt_tour_length": opt_tour_length,
        }

        best_tour = None
        best_tour_length = None
        best_tour_k = None

        best_sa_tour = None
        best_sa_tour_length = None
        best_sa_tour_k = None
        
        c_tour = None
        c_tour_length = None

        start_time = time.time()

        for k in range(1, 20):
            print(name, k) 
            tree = st.get_mst_k(G, k)
            tour = christofides(G, tree=tree)
            tour_length = get_tour_length(G, tour)
            
            if k == 1:
                c_tour = tour
                c_tour_length = tour_length

            if best_tour is None or tour_length < best_tour_length:
                best_tour = tour
                best_tour_length = tour_length
                best_tour_k = k

            sa = SimulateAnnealing(
                graph=G,
                initial_solution=tour,
                max_iterations=100000,
                initial_temperature=10000,
                cooling_rate=0.01,
            )
            sa_tour, sa_tour_cost = sa.run()
            sa_tour_length = get_tour_length(G, sa_tour)
            assert sa_tour_cost == sa_tour_length

            if best_sa_tour is None or sa_tour_length < best_sa_tour_length:
                best_sa_tour = sa_tour
                best_sa_tour_length = sa_tour_length
                best_sa_tour_k = k

        end_time = time.time()
        elapsed_time = end_time - start_time
        
        c_apx_ratio = c_tour_length / opt_tour_length
        best_apx_ratio = best_tour_length / opt_tour_length
        best_sa_apx_ratio = best_sa_tour_length / opt_tour_length

        d["c_tour_length"] = c_tour_length
        d["best_tour_length"] = best_tour_length
        d["best_sa_tour_length"] = best_sa_tour_length
        d["c_apx_ratio"] = c_apx_ratio
        d["best_apx_ratio"] = best_apx_ratio
        d["best_sa_apx_ratio"] = best_sa_apx_ratio
        d["best_tour_k"] = best_tour_k
        d["best_sa_tour_k"] = best_sa_tour_k
        d["elapsed_time"] = elapsed_time

        print(list(d.values()))
        writer.writerow(d.values())
        csvfile.flush()

        print()

burma14
burma14 1
burma14 2
burma14 3
burma14 4
burma14 5
burma14 6
burma14 7
burma14 8
burma14 9
burma14 10
burma14 11
burma14 12
burma14 13
burma14 14
burma14 15
burma14 16
burma14 17
burma14 18
burma14 19
['burma14', 14, 3323, 3559, 3413, 3323, 1.0710201625037616, 1.0270839602768582, 1.0, 14, 6, 37.68416118621826]

ulysses16
ulysses16 1
ulysses16 2
ulysses16 3
ulysses16 4
ulysses16 5
ulysses16 6
ulysses16 7
ulysses16 8
ulysses16 9
ulysses16 10
ulysses16 11
ulysses16 12
ulysses16 13
ulysses16 14
ulysses16 15
ulysses16 16
ulysses16 17
ulysses16 18
ulysses16 19
['ulysses16', 16, 6859, 6982, 6982, 6859, 1.0179326432424551, 1.0179326432424551, 1.0, 1, 7, 43.282649755477905]

gr17
gr17 1
gr17 2
gr17 3
gr17 4
gr17 5
gr17 6
gr17 7
gr17 8
gr17 9
gr17 10
gr17 11
gr17 12
gr17 13
gr17 14
gr17 15
gr17 16
gr17 17
gr17 18
gr17 19
['gr17', 17, 2085, 2211, 2164, 2085, 1.0604316546762589, 1.0378896882494004, 1.0, 6, 12, 41.66158652305603]

gr21
gr21 1
gr21 2
gr21 3
gr21 4
gr21 5
gr21 6
gr21 7
gr21 8


KeyboardInterrupt: 